In [13]:
import sqlite3
from pathlib import Path
from typing import Any, Dict, List

import numpy as np
import pandas as pd
import torch
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.profiler import SimpleProfiler
from sklearn.model_selection import KFold
from torch.optim import SGD
from tqdm import tqdm

from graphnet.data.constants import FEATURES, TRUTH
from graphnet.models import StandardModel
from graphnet.models.detector.icecube import IceCubeKaggle
from graphnet.models.gnn import DynEdge
from graphnet.models.graph_builders import KNNGraphBuilder
from graphnet.models.task.reconstruction import (
    DirectionReconstructionWithKappa,
)
from graphnet.training.callbacks import PiecewiseLinearLR, ProgressBar
from graphnet.training.labels import Direction
from graphnet.training.loss_functions import VonMisesFisher3DLoss
from graphnet.training.utils import make_dataloader
from graphnet.utilities.logging import get_logger
from icecube.constants import *


torch.set_float32_matmul_precision("high")

PULSEMAP = "pulse_table"
DATABASE_PATH = database_dir / "batch_51_100.db"
# DATABASE_PATH = "/media/eden/sandisk/projects/icecube/input/sqlite/batch_1.db"
PULSE_THRESHOLD = 400
SEED = 42

# Training configs
MAX_EPOCHS = 100
LR = 5e-4
MOMENTUM = 0.9
BS = 256
ES = 10
NUM_FOLDS = 5
NUM_WORKERS = 4

# Paths
FOLD_PATH = input_dir / "folds"
COUNT_PATH = FOLD_PATH / "batch51_100_counts.csv"
CV_PATH = FOLD_PATH / f"batch51_100_cv_max_{PULSE_THRESHOLD}_pulses.csv"
WANDB_DIR = log_dir
PROJECT_NAME = "icecube"
GROUP_NAME = "pretrain_sub_5_batch_51_100_large_resume"

CREATE_FOLDS = False


def make_selection(
    df: pd.DataFrame, num_folds: int = 5, pulse_threshold: int = 200
) -> None:
    """Creates a validation and training selection (20 - 80). All events in both selections satisfies n_pulses <= 200 by default."""
    n_events = np.arange(0, len(df), 1)
    df["fold"] = 0

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=SEED)
    for i, (_, val_idx) in enumerate(kf.split(n_events)):
        df.loc[val_idx, "fold"] = i

    # Remove events with large pulses from training and validation sample (memory)
    df["fold"][df["n_pulses"] > pulse_threshold] = -1

    df.to_csv(CV_PATH)
    return


def get_number_of_pulses(db: Path, event_id: int, pulsemap: str) -> int:
    with sqlite3.connect(str(db)) as con:
        query = f"select event_id from {pulsemap} where event_id = {event_id} limit 20000"
        data = con.execute(query).fetchall()
    return len(data)


def count_pulses(database: Path, pulsemap: str) -> pd.DataFrame:
    """Will count the number of pulses in each event and return a single dataframe that contains counts for each event_id."""
    with sqlite3.connect(str(database)) as con:
        query = "select event_id from meta_table"
        events = pd.read_sql(query, con)
    counts = {"event_id": [], "n_pulses": []}

    for event_id in tqdm(events["event_id"]):
        a = get_number_of_pulses(database, event_id, pulsemap)
        counts["event_id"].append(event_id)
        counts["n_pulses"].append(a)

    df = pd.DataFrame(counts)
    df.to_csv(COUNT_PATH)
    return df


def build_model(
    config: Dict[str, Any], train_dataloader: Any = None
) -> StandardModel:
    """Builds GNN from config"""
    # Building model
    detector = IceCubeKaggle(
        graph_builder=KNNGraphBuilder(nb_nearest_neighbours=8),
    )
    gnn = DynEdge(
        nb_inputs=detector.nb_outputs,
        global_pooling_schemes=["min", "max", "mean"],
        dynedge_layer_sizes=config.get("layer_sizes"),
    )
    if config.get("activation"):
        gnn._activation = config["activation"]()

    if config["target"] == "direction":
        task = DirectionReconstructionWithKappa(
            hidden_size=gnn.nb_outputs,
            target_labels=config["target"],
            loss_function=VonMisesFisher3DLoss(),
        )
        prediction_columns = [
            config["target"] + "_x",
            config["target"] + "_y",
            config["target"] + "_z",
            config["target"] + "_kappa",
        ]
        additional_attributes = ["zenith", "azimuth", "event_id"]

    model = StandardModel(
        detector=detector,
        gnn=gnn,
        tasks=[task],
        optimizer_class=SGD,
        optimizer_kwargs={
            "lr": LR,
            "momentum": MOMENTUM,
            "nesterov": True,
        },
    )
    model.prediction_columns = prediction_columns
    model.additional_attributes = additional_attributes

    return model


def load_pretrained_model(
    config: Dict[str, Any],
    state_dict_path: str = "/kaggle/input/dynedge-pretrained/dynedge_pretrained_batch_1_to_50/state_dict.pth",
) -> StandardModel:
    # train_dataloader, _ = make_dataloaders(config=config)
    model = build_model(config=config, train_dataloader=None)
    # model._inference_trainer = Trainer(config['fit'])
    try:
        model.load_state_dict(state_dict_path)
    except:
        state_dict = torch.load(state_dict_path)["state_dict"]
        model.load_state_dict(state_dict)

    model.prediction_columns = [
        config["target"] + "_x",
        config["target"] + "_y",
        config["target"] + "_z",
        config["target"] + "_kappa",
    ]
    model.additional_attributes = ["zenith", "azimuth", "event_id"]
    return model


def make_dataloaders(config: Dict[str, Any], fold: int = 0) -> List[Any]:
    """Constructs training and validation dataloaders for training with early stopping."""
    df_cv = pd.read_csv(CV_PATH)

    val_idx = (
        df_cv[df_cv["fold"] == fold][config["index_column"]].ravel().tolist()
    )
    train_idx = (
        df_cv[~df_cv["fold"].isin([-1, fold])][config["index_column"]]
        .ravel()
        .tolist()
    )

    train_dataloader = make_dataloader(
        db=config["path"],
        selection=train_idx,
        pulsemaps=config["pulsemap"],
        features=FEATURES.KAGGLE,
        truth=TRUTH.KAGGLE,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        shuffle=True,
        labels={"direction": Direction()},
        index_column=config["index_column"],
        truth_table=config["truth_table"],
    )

    validate_dataloader = make_dataloader(
        db=config["path"],
        selection=val_idx,
        pulsemaps=config["pulsemap"],
        features=FEATURES.KAGGLE,
        truth=TRUTH.KAGGLE,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        shuffle=False,
        labels={"direction": Direction()},
        index_column=config["index_column"],
        truth_table=config["truth_table"],
    )

    return train_dataloader, validate_dataloader


def convert_to_3d(df: pd.DataFrame) -> pd.DataFrame:
    """Converts zenith and azimuth to 3D direction vectors"""
    df["true_x"] = np.cos(df["azimuth"]) * np.sin(df["zenith"])
    df["true_y"] = np.sin(df["azimuth"]) * np.sin(df["zenith"])
    df["true_z"] = np.cos(df["zenith"])
    return df


def calculate_angular_error(df: pd.DataFrame) -> pd.DataFrame:
    """Calcualtes the opening angle (angular error) between true and reconstructed direction vectors"""
    df["angular_error"] = np.arccos(
        df["true_x"] * df["direction_x"]
        + df["true_y"] * df["direction_y"]
        + df["true_z"] * df["direction_z"]
    )
    return df


def inference(model, config: Dict[str, Any]) -> pd.DataFrame:
    """Applies model to the database specified in config['inference_database_path'] and saves results to disk."""
    # Make Dataloader
    test_dataloader = make_dataloader(
        db=config["inference_database_path"],
        selection=None,
        pulsemaps=config["pulsemap"],
        features=config["features"],
        truth=config["truth"],
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        shuffle=False,
        labels=None,  # Cannot make labels in test data
        index_column=config["index_column"],
        truth_table=config["truth_table"],
    )

    # Get predictions
    results = model.predict_as_dataframe(
        gpus=[0],
        dataloader=test_dataloader,
        prediction_columns=model.prediction_columns,
        additional_attributes=model.additional_attributes,
    )
    return results


def prepare_dataframe(
    df, angle_post_fix="_reco", vec_post_fix=""
) -> pd.DataFrame:
    r = np.sqrt(
        df["direction_x" + vec_post_fix] ** 2
        + df["direction_y" + vec_post_fix] ** 2
        + df["direction_z" + vec_post_fix] ** 2
    )
    df["zenith" + angle_post_fix] = np.arccos(
        df["direction_z" + vec_post_fix] / r
    )
    df["azimuth" + angle_post_fix] = np.arctan2(
        df["direction_y" + vec_post_fix], df["direction_x" + vec_post_fix]
    )  # np.sign(results['true_y'])*np.arccos((results['true_x'])/(np.sqrt(results['true_x']**2 + results['true_y']**2)))
    df["azimuth" + angle_post_fix][df["azimuth" + angle_post_fix] < 0] = (
        df["azimuth" + angle_post_fix][df["azimuth" + angle_post_fix] < 0]
        + 2 * np.pi
    )

    return df


def angular_dist_score(
    az_true: np.ndarray,
    zen_true: np.ndarray,
    az_pred: np.ndarray,
    zen_pred: np.ndarray,
) -> np.ndarray:
    """
    Calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse
    cosine (arccos) thereof is then the angle between the two input vectors

    Args:
        az_true  (np.ndarray): true azimuth value(s) in radian
        zen_true (np.ndarray): true zenith value(s) in radian
        az_pred  (np.ndarray): predicted azimuth value(s) in radian
        zen_pred (np.ndarray): predicted zenith value(s) in radian

    Returns:
        float: mean over the angular distance(s) in radian
    """

    if not (
        np.all(np.isfinite(az_true))
        and np.all(np.isfinite(zen_true))
        and np.all(np.isfinite(az_pred))
        and np.all(np.isfinite(zen_pred))
    ):
        raise ValueError("All arguments must be finite")

    # Pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)

    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)

    # Scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1 * sz2 * (ca1 * ca2 + sa1 * sa2) + (cz1 * cz2)

    # Scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # That might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod = np.clip(scalar_prod, -1, 1)

    # Convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))


In [14]:
config1 = {
    "inference_database_path": "../../input/icecube/sqlite/test_batch_641_660.db",
    "path": str(DATABASE_PATH),
    "pulsemap": "pulse_table",
    "truth_table": "meta_table",
    "features": FEATURES.KAGGLE,
    "truth": TRUTH.KAGGLE,
    "index_column": "event_id",
    "batch_size": BS,
    "num_workers": NUM_WORKERS,
    "target": "direction",
    "run_name_tag": "batch_1_50",
    "early_stopping_patience": ES,
    "fit": {
        "max_epochs": MAX_EPOCHS,
        "gpus": [0],
        "distribution_strategy": None,
        "limit_train_batches": 0.1,  # debug
        "limit_val_batches": 0.1,
    },
    "base_dir": "training",
    "wandb": {
        "project": PROJECT_NAME,
        "group": GROUP_NAME,
    },
    "lr": LR,
}


config2 = {
    "inference_database_path": "../../input/icecube/sqlite/test_batch_641_660.db",
    "path": str(DATABASE_PATH),
    "pulsemap": "pulse_table",
    "truth_table": "meta_table",
    "features": FEATURES.KAGGLE,
    "truth": TRUTH.KAGGLE,
    "index_column": "event_id",
    "batch_size": BS,
    "num_workers": NUM_WORKERS,
    "target": "direction",
    "run_name_tag": "batch_1_50",
    "early_stopping_patience": ES,
    "fit": {
        "max_epochs": MAX_EPOCHS,
        "gpus": [0],
        "distribution_strategy": None,
        "limit_train_batches": 0.1,  # debug
        "limit_val_batches": 0.1,
    },
    "base_dir": "training",
    "wandb": {
        "project": PROJECT_NAME,
        "group": GROUP_NAME,
    },
    "lr": LR,
    "activation": torch.nn.Mish,
    "layer_sizes": 
        [
                    (
                        128,
                        256,
                    ),
                    (
                        336,
                        256,
                    ),
                    (
                        336,
                        256,
                        256,
                    ),
                    (
                        336,
                        256,
                        256,
                    ),
                ],
}


In [15]:
ckpt1 = "../../models/graphnet/batch1_50/state_dict.pth"
ckpt2 = "/media/eden/sandisk/projects/icecube/logs/icecube/eqk4nmi1/checkpoints/graphnet-val/mae=1.0763-epoch=21.ckpt"

In [16]:
model1 = load_pretrained_model(config1, state_dict_path=ckpt1)
result1 = inference(model1, config1)

model2 = load_pretrained_model(config2, state_dict_path=ckpt2)
result2 = inference(model2, config2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f91b88edcb0>
Traceback (most recent call last):
  File "/home/eden/anaconda3/envs/icecube/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/home/eden/anaconda3/envs/icecube/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/home/eden/anaconda3/envs/icecube/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'

AssertionError: can only test a child processException ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f91b88edcb0>
Traceback (most recent call last):
  File "/home/eden/anaconda3/envs/icecube/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/home/eden/anaconda3/envs/icecube/lib/python3.7/s

TypeError: object of type 'NoneType' has no len()

In [ ]:
test_size = 1_000_000

In [ ]:
results1 = calculate_angular_error(convert_to_3d(result1))
results2 = calculate_angular_error(convert_to_3d(result2))

print("MAE of model 1: ", np.round(results1.iloc[-test_size:]["angular_error"].mean(), 5))
print("MAE of model 2: ", np.round(results2.iloc[-test_size:]["angular_error"].mean(), 5))

/home/eden/anaconda3/envs/icecube/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


MAE of model 1:  1.02621
MAE of model 2:  1.01082


In [ ]:
results1 = prepare_dataframe(results1)
results2 = prepare_dataframe(results2)

print(angular_dist_score(results1["azimuth"], results1["zenith"], results1["azimuth_reco"], results1["zenith_reco"]),
angular_dist_score(results2["azimuth"], results2["zenith"], results2["azimuth_reco"], results2["zenith_reco"]))

1.0260052559287693 1.0104395380336963


In [ ]:
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [ ]:
col_fea = ["direction_x", "direction_y", "direction_z"]
col_tar = ["true_x", "true_y", "true_z"]

In [ ]:
X = np.concatenate([results1[col_fea], results2[col_fea]], axis=1)
y = results1[col_tar].values

X_train = X[:-test_size]
y_train = y[:-test_size]
X_test = X[-test_size:]
y_test = y[-test_size:]

rgr = xgb.XGBRegressor(n_estimators=100, max_depth=4)
rgr.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [ ]:
preds = rgr.predict(X_test)
preds = preds / np.linalg.norm(preds, axis=1).reshape(-1, 1)

ensem_results = results1.iloc[-test_size:].copy()
ensem_results[col_fea] = preds
ensem_results = calculate_angular_error(convert_to_3d(ensem_results))
print("MAE of ensemble: ", np.round(ensem_results["angular_error"].mean(),2))


MAE of ensemble:  1.02


In [ ]:
weighted_results = results1.copy()
weighted_results[col_fea] = results1[col_fea].values * 0.7 + results2[col_fea].values * 0.3
weighted_results[col_fea] /= np.linalg.norm(weighted_results[col_fea], axis=1).reshape(-1, 1)
weighted_results = calculate_angular_error(convert_to_3d(weighted_results))

# print("MAE of ensemble: ", np.round(weighted_results.iloc[-test_size:]["angular_error"].mean(),5))
weighted_results = prepare_dataframe(weighted_results)
angular_dist_score(weighted_results["azimuth"], weighted_results["zenith"], weighted_results["azimuth_reco"], weighted_results["zenith_reco"])

1.0140570780150844